# Yearly Clustering

## Importing

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import BisectingKMeans
from sklearn_som.som import SOM

from tqdm import tqdm


## Datasets Preparation

In [ ]:
def datasets_preparation(data, coords):

    clusters = xr.DataArray(data=data, coords=coords, dims = ['time_counter', 'y', 'x'],
        attrs=dict(description="Clusters produced by the k-means algorithm",
        long_name ="Cluster",
        units="count"))
        
    return (clusters)


## Clustering

In [ ]:
def clustering(dataset, name):

    variable = np.ravel(dataset[name])

    # Pre processing 
    indx = np.where(~np.isnan(variable))
    inputs = variable[indx]
    inputs = inputs.reshape(len(inputs),1)

    kmeans = KMeans(n_clusters=6).fit(inputs)
    predictions = kmeans.predict(inputs)

    # The index to sort the clusters
    indx3 = np.argsort(np.argsort(np.squeeze(kmeans.cluster_centers_, axis=1))) # For the complete map we need the double np.argsort

    # Sorting
    for j in np.arange(0,len(np.unique(predictions))):
        predictions = xr.where(kmeans.labels_==j, indx3[j], predictions)

    # Post processing
    indx2 = np.full(variable.size,np.nan)
    indx2[indx] = predictions
    clusters = np.reshape(indx2,(len(dataset.time_counter),len(dataset.y),len(dataset.x))) 

    return(clusters)
    

## File Creation

In [ ]:
def file_creation(variable, name):

    temp = variable.to_dataset(name=name)
    temp.to_netcdf(path='/data/ibougoudis/MOAD/files/feb_apr_yearly_clustering.nc', mode='a', encoding={name:{"zlib": True, "complevel": 9}})
    temp.close()


## Main Body

In [ ]:
ds = xr.open_dataset('/data/ibougoudis/MOAD/files/feb_apr_c.nc')

coords = dict(time_counter=ds.time_counter, y=ds.y, x=ds.x) 

years = np.unique(ds.time_counter.dt.year)

# ds = ds.isel(
#     y=(np.arange(ds.y[0], ds.y[-1], 5)), 
#     x=(np.arange(ds.x[0], ds.x[-1], 5)))

names = list(ds.keys())
del names[names.index('Summation_of_solar_radiation'):names.index('Day_of_year')+1] # These are continuous 

for name in tqdm(names):
 
    for i in years: 

        dataset = ds.sel(time_counter = slice(str(i), str(i)))     
        clusters = clustering(dataset, name)

        if i == 2007:
            clusters_all = clusters

        else:
            clusters_all = np.append(clusters_all, clusters, axis=0)

    clusters_all = datasets_preparation(clusters_all, coords)

    # Calling file creation
    file_creation(clusters_all, name)
